In [382]:
import pandas as pd 

# 1. upload data
# upload data sets, and convert to pandas data frames
country_stats_data = pd.read_csv("country_happiness_report.csv")
all_int_football_results = pd.read_csv("all_int_football_results.csv")

# other datasets
historic_wc_top_three = pd.read_csv("historical_world_cup_top_three.csv")
fifa_players_data = pd.read_csv("fifa_players.csv")
# historic_wc_games_data = pd.read_csv("historical_world_cup_final_bracket_games.csv")

# 2. clean data.
# replace missing values by mean of set, given attribute and data set.
# replace missing values by mode of set, given attribute and data set.

# given attrs to keep in the set, trim set.

# build mapping based on country selectors from sets.

# merge sets, given attributes and sets.


In [393]:
top_player_attrs = [
    'full_name',
    'nationality',
    'overall_rating',
    'positions',
    'value_euro',
    'wage_euro',
    'preferred_foot',
    'international_reputation(1-5)',
    'body_type',
    'national_team', # use nationality if na
]
top_player_fill_mean_attrs = [
    'overall_rating',
    'value_euro',
    'wage_euro',
    'international_reputation(1-5)',
]
top_player_fill_mode_attrs = [
    'preferred_foot',
    'body_type',
]

selected_attrs_cols = [col for col in fifa_players_data.columns if col in top_player_attrs]
fifa_top_players_data = fifa_players_data[selected_attrs_cols]
fifa_top_players_data['national_team'] = fifa_players_data.national_team.fillna(fifa_top_players_data['nationality'])
# data cleaning - fill by means and modes based on attr set.
fifa_top_players_data[top_player_fill_mean_attrs] = fifa_top_players_data[top_player_fill_mean_attrs].transform(lambda x: x.fillna(x.mean()))
fifa_top_players_data[top_player_fill_mode_attrs] = fifa_top_players_data[top_player_fill_mode_attrs].transform(lambda x: x.fillna(x.mode()))


# probably normalize these
top_player_subset_attrs = [
    'full_name',
    'nationality',
    'overall_rating',
    'crossing',
    'finishing',
    'heading_accuracy',
    'short_passing',
    'volleys',
    'dribbling',
    'curve',
    'freekick_accuracy',
    'long_passing',
    'ball_control',
    'acceleration',
    'sprint_speed',
    'agility',
    'reactions',
    'shot_power',
    'jumping',
    'stamina',
    'strength',
    'long_shots'
]
# normalize skill data and create a subset data table for player skills
top_player_skill_attrs = [attr for attr in top_player_subset_attrs if attr is not 'full_name' and attr is not 'nationality' ]
selected_attrs_cols = [col for col in fifa_players_data.columns if col in top_player_subset_attrs]

fifa_top_players_skills_data = fifa_players_data[selected_attrs_cols]
fifa_top_players_skills_data[top_player_skill_attrs] = fifa_top_players_skills_data[top_player_skill_attrs].apply(lambda x:(x-x.min()) / (x.max()-x.min()))
# mean, mode, normalize
# fifa_top_players_data
fifa_top_players_skills_data.sort_values(by=['overall_rating'], inplace=True, ascending=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,full_name,nationality,overall_rating,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,...,ball_control,acceleration,sprint_speed,agility,reactions,shot_power,jumping,stamina,strength,long_shots
0,Lionel Andrés Messi Cuccittini,Argentina,1.000000,0.920455,1.000000,0.733333,0.988372,0.954023,1.000000,0.988636,...,1.000000,0.929412,0.880952,0.964706,0.986111,0.892473,0.6625,0.705882,0.597403,1.000000
1,Christian Dannemann Eriksen,Denmark,0.872340,0.943182,0.849462,0.533333,0.976744,0.885057,0.860215,0.909091,...,0.945055,0.752941,0.726190,0.811765,0.888889,0.881720,0.4375,0.941176,0.493506,0.945055
2,Paul Pogba,France,0.872340,0.852273,0.784946,0.788889,0.918605,0.942529,0.892473,0.897727,...,0.934066,0.694118,0.797619,0.764706,0.805556,0.946237,0.8500,0.894118,0.870130,0.868132
3,Lorenzo Insigne,Italy,0.872340,0.920455,0.806452,0.577778,0.906977,0.816092,0.924731,0.920455,...,0.967033,0.964706,0.880952,0.976471,0.819444,0.784946,0.4750,0.741176,0.311688,0.890110
4,Kalidou Koulibaly,Senegal,0.872340,0.284091,0.215054,0.877778,0.709302,0.126437,0.698925,0.250000,...,0.637363,0.682353,0.750000,0.458824,0.805556,0.569892,0.8250,0.741176,0.961039,0.131868
5,Virgil van Dijk,Netherlands,0.872340,0.545455,0.537634,0.877778,0.837209,0.482759,0.709677,0.613636,...,0.780220,0.729412,0.773810,0.588235,0.875000,0.849462,0.9125,0.741176,0.935065,0.670330
6,Kylian Mbappé,France,0.872340,0.818182,0.924731,0.811111,0.872093,0.862069,0.924731,0.806818,...,0.945055,0.988235,1.000000,0.952941,0.875000,0.827957,0.7500,0.835294,0.662338,0.824176
7,Sergio Leonel Agüero del Castillo,Argentina,0.893617,0.738636,0.978495,0.811111,0.860465,0.942529,0.913978,0.863636,...,0.923077,0.894118,0.809524,0.882353,0.916667,0.924731,0.8250,0.752941,0.688312,0.879121
8,Manuel Neuer,Germany,0.893617,0.113636,0.118280,0.233333,0.558140,0.091954,0.279570,0.090909,...,0.450549,0.494118,0.571429,0.470588,0.833333,0.247312,0.7750,0.364706,0.779221,0.142857
9,Edinson Roberto Cavani Gómez,Uruguay,0.893617,0.738636,0.935484,0.944444,0.825581,1.000000,0.817204,0.806818,...,0.846154,0.741176,0.761905,0.776471,0.930556,0.913978,0.9125,0.941176,0.753247,0.835165


In [372]:

finishes = ['Winner', 'Runners-Up', 'Third', 'Fourth']

wc_hist_perf_summary = pd.DataFrame()
for finish in finishes:
    wc_hist_perf_summary[finish] = historic_wc_top_three[finish].value_counts()
wc_hist_perf_summary = wc_hist_perf_summary.transform(lambda x: x.fillna(0)).transform(lambda x: x.astype(int))
wc_hist_perf_summary['sum qf appearances'] = a.sum(axis=1)
wc_hist_perf_summary

,Winner,Runners-Up,Third,Fourth,sum qf appearances
Brazil,5,2,2,2,22
Italy,4,2,1,1,16
Germany FR,3,3,1,1,16
Uruguay,2,0,0,3,10
Argentina,2,3,0,0,10
France,1,1,2,1,10
Germany,1,1,3,0,10
England,1,0,0,1,4
Spain,1,0,0,1,4


In [371]:
# minimize, clean, and aggregate country happiness data
# minimize set - 
# country 
# log_gdp_per_capita 
# social_support 
# healthy_life_expectancy_at_birth
# freedom_to_make_life_choices
# generosity
# perceptions_of_corruption
# positive_affect
# negative_affect
# confidence_in_national_government
# democratic_quality
# most_people_can_be_trusted_gallup
selected_attrs_country = [
    'country',
    'year', 
    'Life Ladder',
    'Log GDP per capita',
    'Social support',
    'Healthy life expectancy at birth',
    'Freedom to make life choices',
    'Generosity',
    'Positive affect',
    'Negative affect',
    'Confidence in national government', 
    'Democratic Quality',
    'Most people can be trusted, Gallup',
]

attrs_for_cleaning_nans = [
    'Life Ladder',
    'Log GDP per capita',
    'Social support',
    'Healthy life expectancy at birth',
    'Freedom to make life choices',
    'Generosity',
    'Positive affect',
    'Negative affect',
    'Confidence in national government', 
    'Democratic Quality',
    'Most people can be trusted, Gallup',
]

# minimize data set
selected_attrs_cols = [col for col in country_stats_data.columns if col in selected_attrs_country]
country_data = country_stats_data[selected_attrs_cols]
# set date boundaries 
start_remove = 2014
end_remove = 2016
# group data by country and use mean for group override NAN with mean.
for attr in attrs_for_cleaning_nans:
    country_data[attr] = country_data.groupby('country', as_index=False)[attr].transform(lambda x: x.fillna(x.mean()))

# filter for years in broundaries
country_data = country_data.loc[(country_data.year >= start_remove) & (country_data.year <= end_remove)]
# get mean for country over years
country_data = country_data.groupby('country', as_index=False)[selected_attrs_cols].mean()
# remaining NaNs are because data was NaN for all values in country.  replace these with mean from column.
country_data = country_data.fillna(df.mean())
country_data.head()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Positive affect,Negative affect,Confidence in national government,Democratic Quality,"Most people can be trusted, Gallup"
0,Afghanistan,2015.0,3.777973,7.507917,0.537746,49.588539,0.473336,0.090321,0.550066,0.354156,0.331531,-1.895385,0.281074
1,Albania,2015.0,4.643838,9.251357,0.634452,68.531487,0.722773,-0.041561,0.682841,0.335558,0.468892,0.116744,0.126247
2,Algeria,2015.0,5.871535,9.531784,0.783170,64.624016,0.569640,-0.212148,0.647371,0.274119,0.464820,-1.085674,0.226295
3,Angola,2014.0,3.794838,8.847354,0.754615,45.092995,0.374542,-0.164495,0.578517,0.367864,0.572346,-0.705554,0.226295
4,Argentina,2015.0,6.598489,9.658498,0.909061,67.291908,0.824661,-0.131493,0.852525,0.284971,0.402231,0.166956,0.199123


In [285]:
# minimize, filter, and aggregate international football db for 2018 wc wins.
# filter for FIFA World Cup matches in 2018
fifa_wc_results = all_int_football_results.loc[all_int_football_results['tournament'] == 'FIFA World Cup']
fifa_wc_results_2018 = fifa_wc_results.loc[all_int_football_results['date'].str.contains('2018')]
# fifa_wc_results_2018

def find_winner(row):
    if row.home_score > row.away_score:
        return row.home_team
    elif row.home_score < row.away_score:
        return row.away_team
    else:
        return 'tie'

# TODO: compile this into dataset in csv form https://www.wikiwand.com/en/2018_FIFA_World_Cup_statistics#/Overall_results
# add new column from function given row data
fifa_wc_results_2018['winner'] = fifa_wc_results_2018.apply(lambda row: find_winner(row), axis=1)
fifa_wc_results_2018_winners = fifa_wc_results_2018.groupby(['winner']).winner.agg('count').to_frame('num_wins').reset_index()
#fifa_wc_results_2018
fifa_wc_results_2018_winners = pd.DataFrame(fifa_wc_results_2018_winners)
fifa_wc_results_2018_winners.sort_values(by=['num_wins'], inplace=True, ascending=False)
fifa_wc_results_2018_winners.head()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,winner,num_wins
26,tie,13
1,Belgium,6
7,France,6
4,Croatia,4
25,Uruguay,4


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
39861,2018-06-14,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia,False,Russia
39862,2018-06-15,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia,True,Uruguay
39863,2018-06-15,Morocco,Iran,0,1,FIFA World Cup,St. Petersburg,Russia,True,Iran
39864,2018-06-15,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia,True,tie
39865,2018-06-16,France,Australia,2,1,FIFA World Cup,Kazan,Russia,True,France


In [373]:
# join simple soccer wins and country data set
merged_wins_country = pd.merge(left=fifa_wc_results_2018_winners,right=country_data, left_on='winner', right_on='country')
merged_wins_country = merged_wins_country.drop(['winner'], axis=1)
merged_wins_country

,num_wins,country,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Positive affect,Negative affect,Confidence in national government,Democratic Quality,"Most people can be trusted, Gallup"
0,6,Belgium,2015.0,6.902828,10.624256,0.919241,71.521675,0.865396,-0.006026,0.789134,0.250390,0.449104,1.075157,0.306121
1,6,France,2015.0,6.433234,10.527748,0.886049,72.462580,0.802430,-0.126228,0.789894,0.233777,0.292011,0.851397,0.198650
2,4,Croatia,2014.5,5.293065,9.914950,0.707031,67.503502,0.606201,0.013581,0.602532,0.289957,0.259938,0.538784,0.206280
3,4,Uruguay,2015.0,6.453670,9.900286,0.897924,68.120193,0.902528,-0.072871,0.867820,0.278072,0.523013,0.990709,0.277648
4,3,Sweden,2015.0,7.298938,10.718898,0.924747,71.961449,0.932794,0.172794,0.823103,0.199762,0.516122,1.371415,0.562753
5,3,Brazil,2015.0,6.634238,9.576959,0.905822,64.589249,0.773107,-0.088033,0.768845,0.300108,0.272635,0.178093,0.144043
6,2,Mexico,2015.0,6.580097,9.711609,0.812024,67.803159,0.750071,-0.137134,0.801859,0.228497,0.287240,-0.333587,0.275570
7,2,Russia,2015.0,5.962487,10.088440,0.922349,62.207497,0.714464,-0.218611,0.667556,0.141283,0.624568,-0.910483,0.246937
8,2,Colombia,2015.0,6.356692,9.469525,0.893068,63.848180,0.809019,-0.103823,0.826635,0.288016,0.277137,-0.764890,0.134769
9,1,Switzerland,2015.0,7.507820,10.918212,0.941586,72.912051,0.937050,0.075053,0.803638,0.186957,0.779299,1.436665,0.452701
